In [4]:
import pandas as pd
from PIL import Image
from skimage.transform import resize
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from tensorflow.keras import layers, models
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import LeakyReLU
import seaborn as sns
import keras

In [5]:
# weather mapping function
def weatherName(i):
    if (i == 1):
        return "berembun"
    elif (i == 2):
        return "berkabut"
    elif (i == 3):
        return "hujan es"
    elif (i == 4):
        return "petir"
    elif (i == 5):
        return "hujan"
    elif (i == 6):
        return "pelangi"
    elif (i == 7):
        return "badai pasir"
    else:
        return "salju"

# weather code mapping function for training data
def weatherTrain(i):
    if (i < 200):
        return 1
    elif (i < 400):
        return 2
    elif (i < 600):
        return 3
    elif (i < 800):
        return 4
    elif (i < 1000):
        return 5
    elif (i < 1200):
        return 6
    elif (i < 1400):
        return 7
    else:
        return 8

# mapping for data training results (y_train)
y_train = np.array([weatherTrain(index) for index in range(0, 1600)], dtype=int)
y_train.reshape(-1,)
print(y_train.shape, y_train)


(1600,) [1 1 1 ... 8 8 8]


In [6]:
# data image preprocessing
# input training dataset to numpy array
dew = np.array([resize(np.array(Image.open('./Train/dew/' + filename)) ,(150,200,3)) for filename in os.listdir('./Train/dew')], dtype=float)

fogsmog = np.array([resize(np.array(Image.open('./Train/fogsmog/' + filename)) ,(150,200,3)) for filename in os.listdir('./Train/fogsmog')], dtype=float)

join = np.append(dew, fogsmog,  0)

hail = np.array([resize(np.array(Image.open('./Train/hail/' + filename)) ,(150,200,3)) for filename in os.listdir('./Train/hail')], dtype=float)

join = np.append(join, hail,  0)

lightning = np.array([resize(np.array(Image.open('./Train/lightning/' + filename)) ,(150,200,3)) for filename in os.listdir('./Train/lightning')], dtype=float)

join = np.append(join, lightning,  0)

rain = np.array([resize(np.array(Image.open('./Train/rain/' + filename)) ,(150,200,3)) for filename in os.listdir('./Train/rain')], dtype=float)

join = np.append(join, rain,  0)

rainbow = np.array([resize(np.array(Image.open('./Train/rainbow/' + filename)) ,(150,200,3)) for filename in os.listdir('./Train/rainbow')], dtype=float)

join = np.append(join, rainbow,  0)

sandstorm = np.array([resize(np.array(Image.open('./Train/sandstorm/' + filename)) ,(150,200,3)) for filename in os.listdir('./Train/sandstorm')], dtype=float)

join = np.append(join, sandstorm,  0)

snow = np.array([resize(np.array(Image.open('./Train/snow/' + filename)) ,(150,200,3)) for filename in os.listdir('./Train/snow')], dtype=float)

# mapping for data training (X_train)
X_train = np.append(join, snow,  0)

# check data training shape
print(X_train.shape, y_train.shape)

(1600, 150, 200, 3) (1600,)
